# 爬取網站 : [IT邦幫忙](https://ithelp.ithome.com.tw/)
## 抓取內容 : 文章的標題、標籤、詢問日期、瀏覽次數、內文
### **P.S.1 因為mode是使用'a'附加資料的方式寫入，所以使用前請先確認JSONcrawler.json檔案不存在，否則資料會有重疊**
#### P.S.2 標籤及內文為了避免跟JSON檔案格式衝突，因此有使用REGEX替換特定符號，替換前及替換後如下
#### ( (" -> ')　, 　(\ -> BACKSLASH)　,　(換行符號(\n) -> EOF)　,　(Tab(\t) ->     (四個空白)) )

# 函式

In [1]:
import urllib.request as req #引入urllib模組中函式名稱為request的函式為req
import bs4 #引入bs4模組
import re #引入re模組
#定義函式getData，用以取得每一頁所有標題、標籤及內文。參數為網址
def getData(url,count,pages):
    #建立一個Request物件，附加Request Headers的資訊
    request = req.Request(url,headers = {
        "User-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.0.0 Safari/537.36"
    })
    
    #打開request
    with req.urlopen(request) as response:
        data = response.read().decode("utf-8")

    root = bs4.BeautifulSoup(data,"html.parser") #讓BeautifulSoup解析HTML格式文件
    all_titles = root.find_all("h3",class_="qa-list__title") #以列表形式找出所有class_=title的h3標籤


    #開啟finalproject.txt並用a模式保留原本資料並將新的資料寫入
    with open("JSONcrawler.json",mode = "a",encoding = "utf-8") as ftext:
        #此迴圈會找出該頁所有標題、標籤、詢問時間、瀏覽次數及內文
        for title in all_titles:
            title.a.string = re.sub(r"\"","'",title.a.string) #利用regex將 " 改成 ' ，以免語法錯誤
            title.a.string = re.sub(r"\\"," BACKSLASH ",title.a.string) #利用regex將 \ 改成 BACKLASH ，以免語法錯誤
            ftext.write("\n   {\n") #寫入與JSON檔的檔案格式有關的detail
            ftext.write("   \"標題\":"+"\""+title.a.string+"\","+"\n") #將內文的標題寫入JSON檔裡
            print("標題 : "+title.a.string)
            
            ftext.write("   \"標籤\":[") #寫入與JSON檔的檔案格式有關的detail

            url2 = title.a["href"] #將內文的網址存到url2裡

            #建立一個Request物件，附加Request Headers的資訊
            request2 = req.Request(url2,headers = {
                "User-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.0.0 Safari/537.36"
            })

            #打開request2
            with req.urlopen(request2) as response2:
                data2 = response2.read().decode("utf-8")
            
            root2 = bs4.BeautifulSoup(data2,"html.parser") #讓BeautifulSoup解析HTML格式文件
            contents = root2.find("div",class_="markdown__style").text #找出內文
            tags = root2.find("div",class_="qa-header__tagGroup") #以列表形式找出所有 class_=qa-header__tagGroup 的 div 標籤
            time_and_view = root2.find("div",class_="qa-header__info").text.split() #以列表形式找出所有 class_=qa-header__info 的 div 標籤
            
            #利用regex將會影響JSON的符號做替換
            contents = re.sub(r"\"","'",contents)
            contents = re.sub(r"\\"," BACKSLASH ",contents)
            contents = re.sub(r"\n","EOL",contents)
            contents = re.sub(r"\t","    ",contents)

            set_of_tags = set(tags.text.split("\n")) #利用集合將重複出現的標籤強制讓它出現一次
            #將集合內的資料重新裝回list內
            list_of_tags = []
            for i in set_of_tags:
                list_of_tags.append(i)
            
            #將標籤寫進JSON檔
            for tag in list_of_tags[1:]:
                tag = re.sub(r"\\"," BACKSLASH ",tag)
                if (tag!=list_of_tags[-1]):
                    ftext.write("\"" + tag + "\",")
                else:
                    ftext.write("\"" + tag + "\"],\n")
            
            print("標籤 :",end = " ")
            for tag in list_of_tags[1:]:
                print(tag,end = " ")
            
            ftext.write("   \"詢問時間\":\""+time_and_view[1]+"\",\n") #將詢問時間寫入JSON檔
            print("\n詢問時間 : "+time_and_view[1])
            ftext.write("   \"瀏覽次數\":\""+time_and_view[-2]+"\",\n") #將瀏覽次數寫入JSON檔
            print("瀏覽次數 : "+time_and_view[-2])

            #將內文寫入JSON檔
            if ( (count==pages-1) and (title.a.string == all_titles[-1].a.string)):
                ftext.write("   \"內文\":\""+contents+"\"\n") #將內文寫入content.txt裡
                ftext.write("   }]\n}")
                print("內文 : "+contents)
            else:
                ftext.write("   \"內文\":\""+contents+"\"\n")
                ftext.write("   },\n")
                print("內文 : "+contents+"\n\n")


    #抓取下一頁的連結
    nextLink = root.find("a",string="下一頁") #找到內文是 下一頁 的標籤a
    return nextLink["href"] #回傳下一頁的網址


# 主程式
### 要求使用者輸入想抓取多少頁數的資料

In [2]:
#寫入與JSON檔的檔案格式有關的detail
with open("JSONcrawler.json",mode = "a",encoding = "utf-8") as ftext:
    ftext.write("{\n")
    ftext.write("   \"Article\":[")
pageURL = "https://ithelp.ithome.com.tw/" #將起始網頁的網址存到pageURL裡
count = 0 #count用來表示想抓取幾頁
pages = int(input("Please enter how many pages you want to crawl:"))
#重複抓取每頁資料
while count<pages:
    pageURL = getData(pageURL,count,pages) #將 return 回來的網址覆蓋到pageURL上，以便重複利用
    count += 1 #執行完一次後增加1，以確保能抓到預期的頁數

Please enter how many pages you want to crawl:1
標題 : 爬蟲問題
標籤 : error csv 爬蟲 文字編碼 big5 
詢問時間 : 2022-11-02
瀏覽次數 : 62
內文 : EOL我原本想用爬蟲把IT邦邦忙網站中的文章'標題'，'標籤'，'詢問時間'，'瀏覽次數'，還有'問題敘述'抓下來用csv檔儲存，禮拜一用的時候好好的，結果今天用就突然掛掉。EOL我看了一下error好像是有人文章裡的'問題敘述'格式不符合csv檔用的文字編碼big5EOL跑出的error長這樣 :EOLUnicodeEncodeError: 'big5' codec can't encode character ' BACKSLASH u6ca1' in position 3244: illegal multibyte sequenceEOL想請問這段error是什麼意思?要如何解決?EOL更:我自己這篇文章的東西有抓下來，到下面那篇文章的'問題敘述'就抓不下來了，應該是那篇文章的內容有影響到EOLcontents = root2.find('div',class_='markdown__style').text #找出內文EOLEOL這段是我用來抓'問題敘述'的程式EOLEOL


標題 : Python , 串接LINE_message_API 疑問
標籤 : python line messaging api api 
詢問時間 : 2022-11-02
瀏覽次數 : 101
內文 : EOL大家好 , 我使用LINE_message_API 串接官方帳號, 想做在官方帳號輸入某些關鍵字,就可以EOL查詢別的API所回傳的數值. 比方說我在LINE 官方帳號中輸入蛋價,此時會讀取蛋價的API後,EOL回傳蛋價的數值到LINE官方帳號給使用者.EOL我不太理解, 我在外面讀取了API後,轉成price 為我要的展示的文字數值,在文字輸入雞蛋EOL內寫入text = print(price) 就是會出現錯誤.... 請問那要用什麼樣的語法才能順利讀取到值在LINE官方帳號中顯示給使用者? 感謝大家回應!EOL如下截圖EOLEOLfrom linebot.models import (EOL    MessageEvent, T

標籤 : xfs linux 
詢問時間 : 2022-11-02
瀏覽次數 : 170
內文 : EOL如題User抱怨某個目錄不夠用希望將其他目錄底下的空間分一點過來，看了一下這台系統很老主要filesystem都是XFS沒法透過LVM玩挖東牆補西牆，所以只能請User買硬碟嗎?還是有其他方法有請各為大大指點!!!EOL


標題 : 儲存/除錯 ajax 的錯誤訊息
標籤 : asp.net mvc javascript ajax error xmlhttprequest 
詢問時間 : 2022-11-02
瀏覽次數 : 160
內文 : EOL目前有使用到 ajax 在error 的時候如下EOL error: function () {EOL                            alert(`wrong`);EOL                        }EOLEOL我有查一般想知道錯訊是這樣寫 如下EOLerror: function(xhr, ajaxOptions, thrownError) {EOLconsole.log(xhr.responseText);EOL}EOLEOL我希望發生此錯誤時EOL能記下來這個錯訊EOL因為我無法模擬出那個錯誤EOL不知道問題在哪EOL大家會怎麼存EOL


標題 : 換主板及硬碟之WIN10授權問題
標籤 : @硬碟 @windows10 @序號 @授權 @win10 
詢問時間 : 2022-11-02
瀏覽次數 : 305
內文 : EOL大家好，小弟最近從友人那邊接手一塊二手主板 + 5顆HDD硬碟回來玩玩EOL遇到兩個狀況：EOL1：EOL朋友已經將原主板上的授權轉移至他的新電腦 (WIN11)EOL但連網後仍不用輸入序號，自動啟用專業版 (數位授權)EOL朋友那邊也都正常沒有異樣，這樣兩組序號不是已經衝突了嗎？怎麼能同時運行在兩台電腦上？EOL2：EOL5顆硬碟拿來時已格式化，我自行用USB灌WIN10 PROEOL連網後同上述，自動啟用專業版(數位授權)，但用Showkeyplus及slmgr指令查詢後發現：EOL硬碟1：OEM 序號與其他顆不同EOL硬碟2、3、4：RETAIL 序號相同EOL硬碟5：RETAIL 序號與其他顆不同EOL想請問大神們這是什麼原因？EOL以

# 確認爬取後的檔案可使用

In [3]:
import json #載入json模組
test = open('JSONcrawler.json',encoding="utf8") #開檔
Data = json.load(test) #反序列化(decode)
#把蒐集到的資料印出來
temp = Data.get('Article') #取得 key = "Title" 的 value 
temp #印出 key = "Title" 的 value

[{'標題': '爬蟲問題',
  '標籤': ['error', 'csv', '爬蟲', '文字編碼', 'big5'],
  '詢問時間': '2022-11-02',
  '瀏覽次數': '62',
  '內文': "EOL我原本想用爬蟲把IT邦邦忙網站中的文章'標題'，'標籤'，'詢問時間'，'瀏覽次數'，還有'問題敘述'抓下來用csv檔儲存，禮拜一用的時候好好的，結果今天用就突然掛掉。EOL我看了一下error好像是有人文章裡的'問題敘述'格式不符合csv檔用的文字編碼big5EOL跑出的error長這樣 :EOLUnicodeEncodeError: 'big5' codec can't encode character ' BACKSLASH u6ca1' in position 3244: illegal multibyte sequenceEOL想請問這段error是什麼意思?要如何解決?EOL更:我自己這篇文章的東西有抓下來，到下面那篇文章的'問題敘述'就抓不下來了，應該是那篇文章的內容有影響到EOLcontents = root2.find('div',class_='markdown__style').text #找出內文EOLEOL這段是我用來抓'問題敘述'的程式EOLEOL"},
 {'標題': 'Python , 串接LINE_message_API 疑問',
  '標籤': ['python', 'line messaging api', 'api'],
  '詢問時間': '2022-11-02',
  '瀏覽次數': '101',
  '內文': "EOL大家好 , 我使用LINE_message_API 串接官方帳號, 想做在官方帳號輸入某些關鍵字,就可以EOL查詢別的API所回傳的數值. 比方說我在LINE 官方帳號中輸入蛋價,此時會讀取蛋價的API後,EOL回傳蛋價的數值到LINE官方帳號給使用者.EOL我不太理解, 我在外面讀取了API後,轉成price 為我要的展示的文字數值,在文字輸入雞蛋EOL內寫入text = print(price) 就是會出現錯誤.... 請問那要用什麼樣的語法才能順利讀取到值在LINE官方帳號中顯示給使用者? 感謝大家回應!EOL如下截圖EOLEOLfrom linebot.models i